## Resources

- savask's code: https://gist.github.com/savask/888aa5e058559c972413790c29d7ad72
- Maya's code: https://github.com/meithecatte/busycoq/blob/master/beaver/src/decider/bouncers.rs

In [1]:
from bbchallenge import *

In [2]:
BOUNCER_ID = 4549643

In [3]:
m = get_machine_i("./all_5_states_undecided_machines_with_global_header",BOUNCER_ID)

In [4]:
pptm(m)

s    0    1
---  ---  ---
A    1RB  0RC
B    1RC  1RE
C    0LD  0LA
D    1RA  1LD
E    ???  1LA


In [1]:
def tape_to_str(tape, pos, tape_head):
    min_pos = min(tape.keys())
    max_pos = max(tape.keys())
    s = ""
    for i in range(min_pos,max_pos+1):
        if i == pos and tape_head == '>':
            s += ">"
        s += "." if tape[i] == 0 else "#"
        if i == pos and tape_head == '<':
            s += "<"
    return s

In [2]:
def TMConfiguration(object):
    def __init__(self, pos, state, step, tape, tape_length, head_direction):
        """
            Args:
                pos: position of the head
                state: current state
                step: current step
                tape: tape
                tape_length: length of the tape
                head_direction: direction of the head (`>`, or `<`)
        """
        self.pos = pos
        self.state = state
        self.step = step
        self.tape = tape
        self.tape_length = tape_length
        self.head_direction = head_direction

    def __str__(self):
        return f"step={self.step}, pos={self.pos}, state={ithl(self.state)},"+\
        f" tape_len={self.tape_length} tape=`{tape_to_str(self.tape, self.pos, self.tape_head)}`"

## Find records

In [3]:
step_limit = 100000

In [53]:
def get_record_breaking_per_step_and_per_length(m, step_limit):
    curr_pos: int = 0
    curr_state: int = 0
    curr_tape = {0:0}
    curr_step = 0
    
    records_per_step = {0: TMConfiguration(0,0,0,curr_tape,1,'>')}
    records_per_length = {0: TMConfiguration(0,0,0,curr_tape,1,'>')}
    
    min_pos = 0
    max_pos = 0
    
    while curr_step < step_limit:
        last_pos = curr_pos
        curr_state, curr_pos = step(m, curr_state, curr_pos, curr_tape)
        curr_step += 1
        
        if curr_pos < min_pos or curr_pos > max_pos:

            tape_length = abs(max(curr_pos,max_pos)-min(curr_pos,min_pos))
            
            records_per_step[curr_step] = TMConfiguration(curr_pos,
                                                        curr_state,
                                                        curr_step,
                                                        curr_tape,
                                                        tape_length,'>' if curr_pos - last_pos > 0 else '<')
            
            records_per_length[tape_length] = TMConfiguration(curr_pos,
                                                                    curr_state,
                                                                    curr_step,
                                                                    curr_tape,
                                                                    tape_length, '>' if curr_pos - last_pos > 0 else '<')
    
        min_pos = min(curr_pos,min_pos)
        max_pos = max(curr_pos,max_pos)

    return records_per_step, records_per_length

In [54]:
records_per_step, records_per_length = get_record_breaking_per_step_and_per_length(m,step_limit)

## Get records breaking triples

We want any triple of record-breaking tapes such that their lengths are in an arithmetic progression and step counts are in a quadratic progression (we test that there's a 4th tape matching the quadratic progression).

In [59]:
def yield_record_breaking_triples(records_per_step, records_per_length):
    """ We want any triple of record-breaking tapes such that their lengths are in an 
    arithmetic progression and step counts are in a quadratic progression (we test that 
    there's a 4th tape matching the quadratic progression)."""
    for len_r1 in records_per_length:
        for len_r2 in records_per_length:
            if len_r1 == len_r2:
                continue
                
            r1 = records_per_length[len_r1]
            r2 = records_per_length[len_r2]
    
            if r1.state != r2.state:
                continue
    
            len_r3 = len_r2 + (len_r2-len_r1)
            
            if len_r3 not in records_per_length:
                continue
    
            r3 = records_per_length[len_r3]
    
            if r3.state != r1.state:
                continue

            if !is_monotonic([r1.pos,r2.pos,r3.pos]):
                continue
    
            a = r2.step - r1.step
            b = r3.step - r2.step
            
            step_r4 = r3.step + 2*b - a
    
            if step_r4 not in records_per_step:
                continue
    
            r4 = records_per_step[step_r4]

            if r4.tape_length != len_r3 + (len_r2-len_r1):
                continue

            if !is_monotonic([r1.pos,r2.pos,r3.pos,r4.pos]):
                continue
    
            if r4.state != r1.state:
                continue
    
            yield (r1,r2,r3)

In [61]:
for i, (r1,r2,r3) in enumerate(yield_record_breaking_triples(records_per_step,records_per_length)):
    print(r1)
    print(r2)
    print(r3)
    print()
    if i >= 3:
        break

step=2, pos=2, state=C, tape_len=2 tape=`##`
step=25, pos=3, state=C, tape_len=5 tape=`#####`
step=76, pos=4, state=C, tape_len=8 tape=`########`

step=2, pos=2, state=C, tape_len=2 tape=`##`
step=76, pos=4, state=C, tape_len=8 tape=`########`
step=262, pos=6, state=C, tape_len=14 tape=`##############`

step=2, pos=2, state=C, tape_len=2 tape=`##`
step=155, pos=5, state=C, tape_len=11 tape=`###########`
step=560, pos=8, state=C, tape_len=20 tape=`####################`

step=2, pos=2, state=C, tape_len=2 tape=`##`
step=262, pos=6, state=C, tape_len=14 tape=`##############`
step=970, pos=10, state=C, tape_len=26 tape=`##########################`

